# How to Render a Metal Design into Ansys

## 1. Perform the necessary imports and create a design in Metal first

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [3]:
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer

QAnsysRenderer.default_options

{'Lj': '10nH',
 'Cj': 0,
 '_Rj': 0,
 'max_mesh_length_jj': '7um',
 'project_path': None,
 'project_name': None,
 'design_name': None,
 'ansys_file_extension': '.aedt',
 'x_buffer_width_mm': 0.2,
 'y_buffer_width_mm': 0.2}

In [4]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

In [5]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this example, the design consists of 4 qubits and 4 CPWs

In [6]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        bus1 = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        bus2 = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'bus2', 'Q2', 'bus1', '6.0 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'bus1', 'Q2', 'bus2', '6.1 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'bus2', 'Q4', 'bus1', '6.0 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'bus1', 'Q4', 'bus2', '6.1 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

## 2. Render into Ansys Q3D
Ansys COM registration occurs automatically.

#### Before running the following cells, setup Ansys Q3D manually:
Manually open a new Ansys project with a Q3D design. Here is how:

1. Launch `ANSYS Electronics Desktop yyyy Rx`.
2. Crate a new Ansys project by clicking on the `New` icon at the top left.
3. Create a new Q3D design inside that project by navigating to the menu `Project` and selecting `Insert Q3D Extractor Design`.

In [7]:
# Access Ansys Q3D renderer
fourq_q3d = design.renderers.q3d

  self.draw()



In [9]:
# Open either a new or existing design based on default options
fourq_q3d.connect_ansys()

INFO 09:24PM [connect_project]: Connecting to Ansys Desktop API...
INFO 09:24PM [load_ansys_project]: 	Opened Ansys App
INFO 09:24PM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 09:24PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/MarcoFacchini/Documents/Ansoft/
	Project:   Project2
INFO 09:24PM [connect_design]: No active design found (or error getting active design). Note: No Design Active
INFO 09:24PM [connect]: 	Connection to Ansys established successfully. 😀 



NOTE: If you need to change the active Ansys Q3D design linked to Metal, first select the correct project in the Ansys GUI by double clicking on it in the leftmost panel. Then re-run the two cells above.

#### Render a single qubit with 2 endcaps

In [12]:
fourq_q3d.render_design(['Q1'], [('Q1', 'readout'), ('Q1', 'bus1'), ('Q1', 'bus2')])

AttributeError: 'NoneType' object has no attribute 'draw_polyline'

## 3. Create a new solution setup in Ansys Q3D and obtain the capacitance matrix
This follows from step 2b above.

In [11]:
# Add a solution setup.
fourq_q3d.add_q3d_setup()
# Example of how to change defaults (just an example. We will not use this setup):
fourq_q3d.add_q3d_setup(name = 'Setup2', max_passes = 20)

Optional keyword parameters and their defaults, listed below, represent parameters in this Q3D solution setup. Running fourq_q3d.add_q3d_setup() is equivalent to going to the Project Manager panel in Ansys, right clicking on Analysis within the active Q3D design, selecting "Add Solution Setup...", and choosing/entering default values in the resulting popup window. To easily keep track of different solution setups, give each of them a different name, otherwise, Ansys will simply append numbers to the end of duplicate names. <br><br>
freq_ghz: float = 5. <br>
name: str = "Setup" <br>
save_fields: bool = False <br>
enabled: bool = True <br>
max_passes: int = 15 <br>
min_passes: int = 2 <br>
min_converged_passes: int = 2 <br>
percent_error: float = 0.5 <br>
percent_refinement: int = 30 <br>
auto_increase_solution_order: bool = True <br>
solution_order: str = 'High' <br>
solver_type: str = 'Iterative <br><br>
These defaults can be modified by including the keywords and their new values inside the parentheses. For example, to increase the maximum number of passes from the default value of 15 up to 20 while leaving all else unchanged, replace the cell above with the following: <br><br>
fourq_q3d.add_q3d_setup(max_passes = 20)

In [ ]:
# Analyze said solution setup.
fourq_q3d.analyze_setup("Setup")

In [ ]:
# Using the analysis results, get its capacitance matrix as a dataframe.
fourq_q3d.get_capacitance_matrix()

The optional keyword parameters of get_capacitance_matrix are given below: <br><br>
variation: str = '' <br>
solution_kind: str = 'AdaptivePass' <br>
pass_number: int = 3

In [ ]:
# Run lumped oscillator model (LOM) simulations and save results in dict_lom.b
dict_lom = fourq_q3d.lumped_oscillator_vs_passes(12.31, 2, 3, 7.0, [0.1, 0.1, 0.1], 9)
dict_lom

Using capacitance matrices obtained from each pass, save the many parameters of the Hamiltonian of the system. lumped_oscillator_vs_passes takes in the following parameters: <br><br>
Lj_nH: float <br>
Cj_fF: float <br>
N: int <br>
fr: Union[list, float] <br>
fb: Union[list, float] <br>
maxPass: int <br>
variation: str = '' <br>
solution_kind: str = 'AdaptivePass' <br>
g_scale: float = 1

In [ ]:
# Calculate and save Hamiltonian parameters from a simulation.
dist_analysis = fourq_q3d.distributed_analysis

dist_analysis allows one to calculate dissipation and other properties. It is used for the convergence plots below.

In [ ]:
fourq_q3d.plot_convergence_main(dist_analysis, dict_lom);
fourq_q3d.plot_convergence_chi(dict_lom)

In [ ]:
fourq_q3d.options

### Code to swap rows and columns in capacitance matrix
from qiskit_metal.analyses.quantization.lumped_capacitive import df_reorder_matrix_basis

df_reorder_matrix_basis(fourq_q3d.get_capacitance_matrix(), 1, 2)